In [1]:
import tensorflow as tf
from tensorflow.keras import models,layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import InceptionResNetV2

2024-07-11 13:22:42.266435: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 13:22:42.266562: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 13:22:42.377104: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
data_dir = '/kaggle/input/disasterdetection/data_disaster_types/train'
val_data_dir = '/kaggle/input/disasterdetection/data_disaster_types/dev'
test_dir = '/kaggle/input/disasterdetection/data_disaster_types/test'

In [3]:
import os



# Get the list of class folders
class_folders = [folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))]

# Iterate through each class folder and count the number of images
for class_folder in class_folders:
    class_path = os.path.join(data_dir, class_folder)
    num_images = len(os.listdir(class_path))
    print(f"Class: {class_folder}, Number of Images: {num_images}")

Class: flood, Number of Images: 2336
Class: hurricane, Number of Images: 1444
Class: other_disaster, Number of Images: 1132
Class: fire, Number of Images: 1270
Class: landslide, Number of Images: 940
Class: earthquake, Number of Images: 2058
Class: not_disaster, Number of Images: 3666


In [4]:
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define input and output directories
input_dir = '/kaggle/input/disasterdetection/data_disaster_types/train'
output_dir = '/kaggle/working/augmented_images'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Copy images from input directory to output directory, excluding 'u.txt'
for class_folder in os.listdir(input_dir):
    if class_folder != 'u.txt' and os.path.isdir(os.path.join(input_dir, class_folder)):
        class_output_dir = os.path.join(output_dir, class_folder)
        os.makedirs(class_output_dir, exist_ok=True)  # Create class output directory
        os.makedirs(os.path.join(class_output_dir, 'subdirectory'), exist_ok=True)  # Create subdirectory
        for filename in os.listdir(os.path.join(input_dir, class_folder)):
            src = os.path.join(input_dir, class_folder, filename)
            dst = os.path.join(class_output_dir, 'subdirectory', filename)
            if os.path.isfile(src) and filename != 'u.txt':
                shutil.copy(src, dst)

In [5]:
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array

# Define output directory
output_dir = '/kaggle/working/augmented_images'

# Find the maximum number of images among all class directories
max_images = 0
for class_folder in os.listdir(output_dir):
    class_folder_path = os.path.join(output_dir, class_folder, 'subdirectory')
    if os.path.isdir(class_folder_path):
        num_images = len(os.listdir(class_folder_path))
        max_images = max(max_images, num_images)

# Perform data augmentation to match the maximum number of images in each class directory
datagen = ImageDataGenerator(
    rotation_range=20,
    
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


for class_folder in os.listdir(output_dir):
    class_folder_path = os.path.join(output_dir, class_folder, 'subdirectory')
    if os.path.isdir(class_folder_path):
        num_images = len(os.listdir(class_folder_path))
        num_images_to_add = max_images - num_images
        i = 0
        if num_images_to_add > 0:
            print(f"Augmenting images for class {class_folder}...")
            for filename in os.listdir(class_folder_path):
                
                if filename.endswith(('.jpg', '.jpeg', '.png')):
                    image_path = os.path.join(class_folder_path, filename)
                    img = load_img(image_path)  # Load image using appropriate method
                    x = img_to_array(img)  # Convert image to numpy array
                    x = x.reshape((1,) + x.shape)  # Reshape to (1, height, width, channels) for flow() method
                    j = 0
                    for batch in datagen.flow(x, batch_size=1, save_to_dir=class_folder_path, save_prefix='augmented', save_format='jpeg'):
                        j += 1
                        i += 1
                        if j == 10:  # Apply augmentation techniques until reaching the target number
                            break
                        if i == num_images_to_add:
                            break
                    if i == num_images_to_add:
                        break  
                    

print("Augmentation completed successfully.")

Augmenting images for class fire...
Augmenting images for class flood...
Augmenting images for class hurricane...
Augmenting images for class earthquake...
Augmenting images for class other_disaster...
Augmenting images for class landslide...
Augmentation completed successfully.


In [6]:
import os
import shutil

# Define the parent directory
parent_directory = "/kaggle/working/augmented_images"

# List of subdirectories to process
subdirectories = ['earthquake','flood','landslide','other_disaster','not_disaster','fire','hurricane']

# Loop through each subdirectory
for subdir in subdirectories:
    subdirectory_path = os.path.join(parent_directory, subdir)

    # List all files in the subdirectory
    files = os.listdir(os.path.join(subdirectory_path, 'subdirectory'))

    # Move each file to the parent directory
    for file in files:
        source = os.path.join(subdirectory_path, 'subdirectory', file)
        destination = os.path.join(subdirectory_path, file)
        shutil.move(source, destination)

    # Remove the now empty subdirectory
    os.rmdir(os.path.join(subdirectory_path, 'subdirectory'))

In [7]:
for class_folder in class_folders:
    num_images = len(os.listdir(f'/kaggle/working/augmented_images/{class_folder}'))
    print(f"Class: {class_folder}, Number of Images: {num_images}")

Class: flood, Number of Images: 3586
Class: hurricane, Number of Images: 3435
Class: other_disaster, Number of Images: 3350
Class: fire, Number of Images: 3416
Class: landslide, Number of Images: 3300
Class: earthquake, Number of Images: 3540
Class: not_disaster, Number of Images: 3666


In [8]:
data_dir = "/kaggle/working/augmented_images"

In [9]:
Imagesize = 224


seed = 123

training_dataset= tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    image_size=(Imagesize,Imagesize),
    batch_size=32,
    shuffle=True,
    seed=seed )

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_data_dir,
    image_size=(Imagesize,Imagesize),
    batch_size=32,
    shuffle=True,
    seed=seed )

test_dataset  = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(Imagesize,Imagesize),
    batch_size=32,
    shuffle=True,
    seed=seed )

Found 24293 files belonging to 7 classes.
Found 1470 files belonging to 7 classes.
Found 3195 files belonging to 7 classes.


In [10]:
rescale = tf.keras.Sequential([
    layers.Rescaling(1./255),
])
data_augmentation = tf.keras.Sequential([
    layers.RandomContrast(0.1),  
    layers.RandomFlip(mode='horizontal_and_vertical'),  
    layers.RandomRotation(factor=0.2),  
    
])

In [11]:

base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model2 = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model3 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [12]:
def create_model(base_model):
    
    model = models.Sequential()

   
    for layer in base_model.layers[:-5]:
        layer.trainable = False

    model = models.Sequential()

   
    model.add(rescale)
    model.add(data_augmentation)

    model.add(base_model)

    
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(7, activation='softmax'))  
    input  =  (32,224,224,3)
    model.build(input_shape = input)
    
    optimizer = Adam(learning_rate=0.0001)

    model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
    )
    
    print(model.summary())

    return model

## InceptionResNetV2


In [13]:
model = create_model(base_model)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (32, 224, 224, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (32, 224, 224, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_resnet_v2             │ (32, 5, 5, 1536)       │    54,336,736 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (32, 1536)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (32, 1536)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 1024)             │     1,573,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (32, 1024)             │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (32, 512)              │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (32, 512)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (32, 128)              │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (32, 7)                │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 57,551,591 (219.54 MB)

 Trainable params: 7,345,191 (28.02 MB)

 Non-trainable params: 50,206,400 (191.52 MB)

None


In [14]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_accuracy',   
    min_delta=0.001,          
    patience=10,              
    verbose=1,               
    mode='max',             
    restore_best_weights=True
)

In [ ]:
record = model.fit(
    training_dataset,
    batch_size=32,
    epochs=40,
    verbose=1,
    validation_data=validation_dataset,
    shuffle=True,
    callbacks=[early_stopping]      
)

Epoch 1/40
760/760 ━━━━━━━━━━━━━━━━━━━━ 208s 237ms/step - accuracy: 0.4107 - loss: 1.5551 - val_accuracy: 0.6395 - val_loss: 0.9975
Epoch 2/40
760/760 ━━━━━━━━━━━━━━━━━━━━ 171s 225ms/step - accuracy: 0.6145 - loss: 1.0809 - val_accuracy: 0.6531 - val_loss: 0.9580
Epoch 3/40
760/760 ━━━━━━━━━━━━━━━━━━━━ 171s 225ms/step - accuracy: 0.6579 - loss: 0.9706 - val_accuracy: 0.6503 - val_loss: 0.9796
Epoch 4/40
760/760 ━━━━━━━━━━━━━━━━━━━━ 171s 225ms/step - accuracy: 0.6882 - loss: 0.8835 - val_accuracy: 0.6633 - val_loss: 0.9416
Epoch 5/40
760/760 ━━━━━━━━━━━━━━━━━━━━ 171s 224ms/step - accuracy: 0.7080 - loss: 0.8464 - val_accuracy: 0.6816 - val_loss: 0.9282
Epoch 6/40
760/760 ━━━━━━━━━━━━━━━━━━━━ 170s 224ms/step - accuracy: 0.7187 - loss: 0.8213 - val_accuracy: 0.6748 - val_loss: 0.9346
Epoch 7/40
760/760 ━━━━━━━━━━━━━━━━━━━━ 170s 224ms/step - accuracy: 0.7350 - loss: 0.7671 - val_accuracy: 0.6789 - val_loss: 0.9100
Epoch 8/40
760/760 ━━━━━━━━━━━━━━━━━━━━ 171s 225ms/step - accuracy: 0.7413 -

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import tensorflow as tf


test_data = []
test_labels = []
for data, labels in test_dataset:
    test_data.append(data.numpy())
    test_labels.append(labels.numpy())

test_data = np.concatenate(test_data)
test_labels = np.concatenate(test_labels)

predictions = model.predict(test_data)
predicted_labels = np.argmax(predictions, axis=1)  # Assuming a classification problem with softmax output
# If labels are one-hot encoded

# Compute the confusion matrix
cm = confusion_matrix(test_labels, predicted_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)

# Visualize the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.show()


## InceptionV3

In [ ]:
model2 = create_model(base_model2)


In [ ]:
record = model2.fit(
    training_dataset,
    batch_size=32,
    epochs=25,
    verbose=1,
    validation_data=validation_dataset,
    shuffle=True,
    callbacks=[early_stopping]      

)

## VGG 16

In [ ]:
model3 = create_model(base_model3)

In [ ]:
record = model3.fit(
    training_dataset,
    batch_size=32,
    epochs=25,
    verbose=1,
    validation_data=validation_dataset,
    shuffle=True,
    callbacks=[early_stopping]      

)

In [ ]:
# loss, accuracy = model.evaluate(test_dataset)
# print(f"Test Loss: {loss}")
# print(f"Test Accuracy: {accuracy}")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator




test_datagen = ImageDataGenerator(rescale=None)  


test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(Imagesize, Imagesize),  
    batch_size=32,
    class_mode='categorical',  
    shuffle=False  
)


In [ ]:
true_classes = test_generator.classes

In [ ]:
predictions = model.predict(test_generator, verbose=1)
predictions2 = model2.predict(test_generator, verbose=1)
predictions3 = model3.predict(test_generator, verbose=1)

### Soft voting

In [ ]:
import numpy as np
average_prob = (predictions + predictions2 + predictions3) / 3

y_pred_soft = np.argmax(average_prob, axis=1)

accuracy_soft = np.mean(y_pred_soft == true_classes)
print('Accuracy:', accuracy_soft)

### Hard Voting

In [ ]:
from scipy.stats import mode

preds = np.argmax(predictions, axis=1)
preds2 = np.argmax(predictions2, axis=1)
preds3 = np.argmax(predictions3, axis=1)
max_voting = np.stack([preds, preds2, preds3], axis=1)

max_voting.shape

In [ ]:
y_pred_hard, _ = mode(max_voting, axis=1)
y_pred_hard = y_pred_hard.ravel()

accuracy = np.mean(y_pred_hard == true_classes)
print('Accuracy:', accuracy)